In [1]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery
import re

In [2]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [3]:
#Nombre de lignes totales dans le JSON : 1320229

In [4]:
#intégration des credentials
from google.oauth2 import service_account

project = "marbotic"
key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [49]:
CHUNKSIZE = 100000
table_id = "marbotic_dataset.user_properties"

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["user_properties"].to_list()).copy()

    for col_concat in ['Products', 'Pieces', 'Games']:
        cat = ep[f'{col_concat}'].copy().map(lambda x: ", ".join(x)
                                            if isinstance(x, list) else x)
        names = list(
            set([
                x.strip().strip("'").strip("['")
                for x in ', '.join(', '.join(', '.join(
                    list(set([str(x) for x in cat]))).split('\n')).split(
                        "' '")).split(',')
            ]))
        names_transf = [
            col_concat + '_' + x.replace(' ', '_') for x in names
            if x != 'nan' and x != ''
        ]
        names = [x for x in names if x != 'nan' and x != '']
        for ind, name in enumerate(names_transf):

            ep[f'{name}'] = ep[f'{col_concat}'].map(
                lambda x: 1 if isinstance(x, str) and f'{names[ind]}' in x else
                (1 if isinstance(x, list) and
                 len([n for n in x if f'{names[ind]}' in n]) > 0
                 else 0))
            #ep['test']=ep[f'{col_concat}'].str.replace('[^\w\s\d]', '', regex=True)

    ep.drop(['Products', 'Pieces', 'Games'], axis=1, inplace=True)
    ep['event_id']=c["event_id"].copy()
    ep['client_event_time']=c['client_event_time'].copy()
    ep['user_creation_time']=c['user_creation_time'].copy()
    ep['user_id']=c['user_id'].copy()
    ep['user_id'] = ep['user_id'].apply(lambda x: 0 if (x is None or type(x)== str or  np.isnan(x)) else int(x))
    ep['id'] = (i*CHUNKSIZE) + index_init
    job = client.load_table_from_dataframe(ep, table_id)
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
        )


Loaded 100000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 200000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 300000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 400000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 500000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 600000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 700000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 800000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 900000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 1000000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 1100000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 1200000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 1300000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 1320229 rows and 40 columns to marbotic_dataset.user_properties


In [ ]:
columns_to_add_manually = ['Products_Sensory_kit'] #integer
query = f"DELETE FROM `{table_id}` WHERE true"
query_job = client.query(query)
results=query_job.result().to_dataframe()
results